In [1]:
import torch
import torch.nn as nn
import torchaudio
import torch.fft
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Audio
import seaborn as sns
import pandas as pd
import os
from torch.nn.utils.rnn import pad_sequence
import torchvision.transforms as transforms
import torchaudio.transforms as transformsaudio
import datetime
from torch.utils.tensorboard import SummaryWriter
import soundfile as sf
import time
import torch.nn.functional as F
import math
import random
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

writer = SummaryWriter()
# %load_ext tensorboard

from prettytable import PrettyTable

def print_count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params += params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

import sys
def size_of_tensor(a):
    return sys.getsizeof(a) + torch.numel(a)*a.element_size()

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

2023-11-22 19:21:44.295616: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-22 19:21:44.295644: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-22 19:21:44.295664: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-22 19:21:44.302408: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-22 19:21:44.952223: W tensorflow/compiler/

In [2]:
directoryBase = os.getcwd()

In [3]:
directoryBase

'/home/afridman/tesisPabloAxel/wavenet'

In [4]:
#import zipfile
#with zipfile.ZipFile('modelos.zip', 'r') as zip_ref:
#    zip_ref.extractall('')

In [5]:
def pad_to_max_length(tensor1, tensor2):
    max_length = max(tensor1.size(1), tensor2.size(1))

    pad_tensor1 = torch.nn.functional.pad(tensor1, (0, max_length - tensor1.size(1)))
    pad_tensor2 = torch.nn.functional.pad(tensor2, (0, max_length - tensor2.size(1)))

    return pad_tensor1, pad_tensor2

In [6]:
inputSize = 32000
class AudioCleaningDataset(Dataset):
    def __init__(self, csv_file, audio_dir, noise_dir, reverb_dir, target_length=inputSize, device='cpu'):
        self.dataframe = pd.read_csv(csv_file)
        self.audio_dir = audio_dir
        self.noise_dir = noise_dir
        self.reverb_dir = reverb_dir
        self.target_length = target_length
        self.resampleo = transformsaudio.Resample(orig_freq=48000, new_freq=16000)  # Resampling
        self.resampleoIR = transformsaudio.Resample(orig_freq=32000, new_freq=16000)  # Resampling

        self.noise_files = os.listdir(noise_dir) # List all noise files
        self.reverb_files = os.listdir(reverb_dir)
        self.device = device
        self.conv= transformsaudio.Convolve(mode="same")


    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        audio_file_name = os.path.join(self.audio_dir, self.dataframe.iloc[idx]['audio_file_name']+".wav")
        waveform, sample_rate = torchaudio.load(audio_file_name)

        waveformOriginal = self.resampleo(waveform)

        #aca seleccionar
        maximoPosible = len(waveformOriginal[0])-self.target_length
        if(maximoPosible>1):
          comienzoAleatorio = (random.randint(0,maximoPosible))
          waveformOriginal = waveformOriginal[:,comienzoAleatorio:comienzoAleatorio+self.target_length]
        #print(len(waveformOriginal[0]))


        waveformSucia = waveformOriginal.clone()
        padding = torch.zeros((1, max(self.target_length - waveformSucia.size(1),1)))

        waveformOriginal = torch.cat((waveformOriginal, padding), dim=1)
        waveformSucia = torch.cat((waveformSucia, padding), dim=1)

        waveformOriginal = waveformOriginal[:,:self.target_length].to(self.device)
        waveformSucia = waveformSucia[:,:self.target_length].to(self.device)

        return 1*waveformSucia, 1*waveformOriginal
    
#         # Load a random noise file
#         noise_file_name = random.choice(self.noise_files)
#         noise_waveform, sample_Rate_ruido = torchaudio.load(os.path.join(self.noise_dir, noise_file_name))
#         # Repeat the noise waveform until it's at least as long as the audio waveform
#         while noise_waveform.size(1) < waveformSucia.size(1):
#             noise_waveform = torch.cat((noise_waveform, noise_waveform), dim=1)

#         # Trim the noise waveform to match the length of the audio waveform
#         noise_waveform = noise_waveform[:,:waveformSucia.size(1)]

#         # Add noise with a random signal-to-noise ratio between 0.01 and 0.1
#         snr = random.uniform(0.001, 0.01) #random.uniform(0.01, 0.1)

#         # LE SACO EL RUIDO SUCIO PARA ACELERAR APRENDIZAJE !!!!!!!!!!!!!!!!!!!!!!!!!!!!
#         #waveformSucia = waveformSucia + noise_waveform * snr
#         whitenoise = random.uniform(0.0001, 0.001)
#         #waveformSucia = waveformSucia + torch.randn_like(waveformOriginal) * whitenoise


#         IR_file_name = random.choice(self.reverb_files)
#         IR_waveform, sample_Rate_IR = torchaudio.load(os.path.join(self.reverb_dir, IR_file_name))
#         IR_waveform = self.resampleoIR(IR_waveform)

#         # Normalize impulse response
#         normalized_ir = IR_waveform / (IR_waveform.abs().max())

#         # Perform convolution
#         padded_signal, padded_filter = pad_to_max_length(waveformSucia, normalized_ir)

#         # Perform the FFT
#         fft_signal = torch.fft.fft(padded_signal)
#         fft_filter = torch.fft.fft(padded_filter)

#         # Perform the convolution in the frequency domain
#         fft_result = fft_signal * fft_filter

#         # Perform the inverse FFT to get the result in the time domain
#         result = torch.fft.ifft(fft_result)

#         # The result is complex, take the real part

#         # LE SACO EL REVERB PARA ACELERAR APRENDIZAJE !!!!!!!!!!!!!!!!!!!!!!!!!!!!
#         #waveformSucia = result.real

#         padding = torch.zeros((1, max(self.target_length - waveformSucia.size(1),1)))

#         waveformOriginal = torch.cat((waveformOriginal, padding), dim=1)
#         waveformSucia = torch.cat((waveformSucia, padding), dim=1)

#         waveformOriginal = waveformOriginal[:,:self.target_length].to(self.device)
#         waveformSucia = waveformSucia[:,:self.target_length].to(self.device)

#        # waveformOriginal = (waveformOriginal - waveformOriginal.mean()) / waveformOriginal.std()
#        # waveformSucia = (waveformSucia - waveformSucia.mean()) / waveformSucia.std()
#        # Optional: Normalize convolved audio
#        # waveformSucia = waveformSucia / (waveformSucia.abs().max())
#        # waveformOriginal = waveformOriginal / (waveformOriginal.abs().max())

#         return 1*waveformSucia, 1*waveformOriginal

In [7]:
"""
Neural network modules for WaveNet

References :
    https://arxiv.org/pdf/1609.03499.pdf
    https://github.com/ibab/tensorflow-wavenet
    https://qiita.com/MasaEguchi/items/cd5f7e9735a120f27e2a
    https://github.com/musyoku/wavenet/issues/4
"""

class DilatedCausalConv1d(torch.nn.Module):
    """Dilated Causal Convolution for WaveNet"""
    def __init__(self, channels, dilation=1, device='cpu'):
        super(DilatedCausalConv1d, self).__init__()

        self.conv = torch.nn.Conv1d(channels, channels,
                                    kernel_size=2, stride=1,  # Fixed for WaveNet
                                    dilation=dilation,
                                    padding=dilation,  # Fixed for WaveNet dilation
                                    bias=False)  # Fixed for WaveNet but not sure
    
        self.conv = self.conv.to(device)

    def init_weights_for_test(self):
        for m in self.modules():
            if isinstance(m, torch.nn.Conv1d):
                m.weight.data.fill_(1)

    def forward(self, x):
        output = self.conv(x)

        return output


class CausalConv1d(torch.nn.Module):
    """Causal Convolution for WaveNet"""
    def __init__(self, in_channels, out_channels, device='cpu'):
        super(CausalConv1d, self).__init__()

        # padding=1 for same size(length) between input and output for causal convolution
        self.conv = torch.nn.Conv1d(in_channels, out_channels,
                                    kernel_size=2, stride=1, padding=1,
                                    bias=False)  # Fixed for WaveNet but not sure
        if torch.cuda.is_available():
            self.conv = self.conv.to(device)

    def init_weights_for_test(self):
        for m in self.modules():
            if isinstance(m, torch.nn.Conv1d):
                m.weight.data.fill_(1)

    def forward(self, x):
        output = self.conv(x)

        # remove last value for causal convolution
        return output[:, :, :-1]


class ResidualBlock(torch.nn.Module):
    def __init__(self, res_channels, skip_channels, dilation, device='cpu'):
        """
        Residual block
        :param res_channels: number of residual channel for input, output
        :param skip_channels: number of skip channel for output
        :param dilation:
        """
        super(ResidualBlock, self).__init__()

        self.dilated = DilatedCausalConv1d(res_channels, dilation=dilation)
        self.conv_res = torch.nn.Conv1d(res_channels, res_channels, 1)
        self.conv_skip = torch.nn.Conv1d(res_channels, skip_channels, 1)

        self.gate_tanh = torch.nn.Tanh()
        self.gate_sigmoid = torch.nn.Sigmoid()

        self.conv_skip = self.conv_skip.to(device)
        self.conv_res = self.conv_res.to(device)

    def forward(self, x, skip_size):
        """
        :param x:
        :param skip_size: The last output size for loss and prediction
        :return:
        """
        output = self.dilated(x)

        # PixelCNN gate
        gated_tanh = self.gate_tanh(output)
        gated_sigmoid = self.gate_sigmoid(output)
        gated = gated_tanh * gated_sigmoid

        # Residual network
        output = self.conv_res(gated)
        output = output[:, :, 0:inputSize]

        input_cut = x#[:, :, -output.size(2):]

        output += input_cut

        # Skip connection
        skip = self.conv_skip(gated)
        skip = skip[:, :, -skip_size:]

        return output, skip


class ResidualStack(torch.nn.Module):
    def __init__(self, layer_size, stack_size, res_channels, skip_channels, device='cpu'):
        """
        Stack residual blocks by layer and stack size
        :param layer_size: integer, 10 = layer[dilation=1, dilation=2, 4, 8, 16, 32, 64, 128, 256, 512]
        :param stack_size: integer, 5 = stack[layer1, layer2, layer3, layer4, layer5]
        :param res_channels: number of residual channel for input, output
        :param skip_channels: number of skip channel for output
        :return:
        """
        super(ResidualStack, self).__init__()

        self.layer_size = layer_size
        self.stack_size = stack_size

        self.device = device
        self.res_blocks = self.stack_res_block(res_channels, skip_channels)

    @staticmethod
    def _residual_block(res_channels, skip_channels, dilation, device='cpu'):
        block = ResidualBlock(res_channels, skip_channels, dilation)

        if torch.cuda.device_count() > 1:
            block = torch.nn.DataParallel(block)

        block.to(device)

        return block

    def build_dilations(self):
        dilations = []

        # 5 = stack[layer1, layer2, layer3, layer4, layer5]
        for s in range(0, self.stack_size):
            # 10 = layer[dilation=1, dilation=2, 4, 8, 16, 32, 64, 128, 256, 512]
            for l in range(0, self.layer_size):
                dilations.append(2 ** l)

        return dilations

    def stack_res_block(self, res_channels, skip_channels):
        """
        Prepare dilated convolution blocks by layer and stack size
        :return:
        """
        res_blocks = []
        dilations = self.build_dilations()

        for dilation in dilations:
            block = self._residual_block(res_channels, skip_channels, dilation, device=self.device)
            res_blocks.append(block)

        return res_blocks

    def forward(self, x, skip_size):
        """
        :param x:
        :param skip_size: The last output size for loss and prediction
        :return:
        """
        output = x
        skip_connections = []

        for res_block in self.res_blocks:
            # output is the next input
            output, skip = res_block(output, skip_size)
            skip_connections.append(skip)

        return torch.stack(skip_connections)


class DensNet(torch.nn.Module):
    def __init__(self, channels, device='cpu'):
        """
        The last network of WaveNet
        :param channels: number of channels for input and output
        :return:
        """
        super(DensNet, self).__init__()

        self.conv1 = torch.nn.Conv1d(channels, channels, kernel_size=3, stride=1, padding=1)
        self.conv2 = torch.nn.Conv1d(channels, channels, kernel_size=3, stride=1, padding=1)

        self.tan = torch.nn.Tanh()

        self.conv1 = self.conv1.to(device)
        self.conv2 = self.conv2.to(device)

    def forward(self, x):
        output = self.conv1(x)
        output = self.tan(output)
        output = self.conv2(output)

        return output

In [8]:
class WaveNet(pl.LightningModule):
    def __init__(self, layer_size, stack_size, in_channels, res_channels):
        """
        Stack residual blocks by layer and stack size
        :param layer_size: integer, 10 = layer[dilation=1, dilation=2, 4, 8, 16, 32, 64, 128, 256, 512]
        :param stack_size: integer, 5 = stack[layer1, layer2, layer3, layer4, layer5]
        :param in_channels: number of channels for input data. skip channel is same as input channel
        :param res_channels: number of residual channel for input, output
        :return:
        """
        super(WaveNet, self).__init__()

        self.receptive_fields = self.calc_receptive_fields(layer_size, stack_size)

        self.causal = CausalConv1d(in_channels, res_channels)

        self.res_stack = ResidualStack(layer_size, stack_size, res_channels, in_channels)

        self.densnet = DensNet(in_channels)

        self.loss_fun = nn.MSELoss()


    @staticmethod
    def calc_receptive_fields(layer_size, stack_size):
        layers = [2 ** i for i in range(0, layer_size)] * stack_size
        num_receptive_fields = np.sum(layers)

        return int(num_receptive_fields)

    def calc_output_size(self, x):
        output_size = int(x.size(2)) - self.receptive_fields

        #self.check_input_size(x, output_size)

        return inputSize

    def check_input_size(self, x, output_size):
        if output_size < 1:
            raise InputSizeError(int(x.size(2)), self.receptive_fields, output_size)

    def forward(self, x):
        """
        The size of timestep(3rd dimention) has to be bigger than receptive fields
        :param x: Tensor[batch, timestep, channels]
        :return: Tensor[batch, timestep, channels]
        """
        output = x#.transpose(1, 2)

        output_size = self.calc_output_size(output)

        output = self.causal(output)

        skip_connections = self.res_stack(output, output_size)


        #en vez de sumar skip connections hago convolucion
        #print(skip_connections.shape)
        output = torch.sum(skip_connections, dim=0)


        output = self.densnet(output)

        return output#.transpose(1, 2).contiguous()

In [9]:
modeloNombre = directoryBase+'/modelos/wavenetReal/wavenetReplicaAudio.pth'

In [10]:
#del wavenetModel
#del dataloader
#del waveform
#torch.cuda.empty_cache()

In [11]:
dataset = AudioCleaningDataset(directoryBase+'/CSV/audio_pocos.csv', directoryBase+'/audiosDivididos/', directoryBase+'/audiosDivididos/ruidosPocos/', directoryBase+"/audiosDivididos/reverbPocos")
dataloader = DataLoader(dataset, batch_size=3, shuffle=True)

In [12]:
waveform, label = next(iter(dataloader))

In [13]:
Audio(data=waveform.cpu()[0], rate=16000)

In [14]:
Audio(data=label.cpu()[0], rate=16000)

In [15]:
#dict(wavenetModel.named_parameters()).keys()

In [16]:
wavenetModel = WaveNet( layer_size=18, stack_size=2, in_channels=1, res_channels=128)

# wavenetModel = torch.load(modeloNombre, map_location=torch.device('cuda'))
wavenetModel = wavenetModel.to('cpu')
#.load_state_dict(torch.load(modeloNombre))
count_parameters(wavenetModel)

264

In [17]:
sum([count_parameters(wavenetModel)]+[count_parameters(rb) for rb in wavenetModel.res_stack.res_blocks])    

1778988

In [18]:
size_of_tensor(waveform)/1024/1024

0.7324905395507812

In [19]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
a/1024/1024/1024, r/1024/1024/1024, t/1024/1024/1024

(0.0, 0.0, 10.9166259765625)

In [21]:
# with torch.no_grad():
wavenetModel(waveform)

tensor([[[-0.2374, -0.1194, -0.1173,  ..., -0.1273, -0.1199, -0.2076]],

        [[-0.2430, -0.1443, -0.1448,  ..., -0.1270, -0.1183, -0.2077]],

        [[-0.2382, -0.1222, -0.1212,  ..., -0.1328, -0.1243, -0.2102]],

        [[-0.2384, -0.1302, -0.1247,  ..., -0.1251, -0.1179, -0.2065]],

        [[-0.2376, -0.1194, -0.1173,  ..., -0.1262, -0.1180, -0.2068]],

        [[-0.2376, -0.1177, -0.1165,  ..., -0.1294, -0.1229, -0.2085]]],
       grad_fn=<ConvolutionBackward0>)

In [34]:
x = list(wavenetModel.causal.parameters())[0]

In [35]:
import gc
del wavenetModel
del waveform
gc.collect()

9441

In [ ]:
#modelPanoramico = WaveNetPanoramico(layer_size=18, stack_size=2, in_channels=1, res_channels=128)
#modelPanoramico.load_state_dict(torch.load(modeloNombre))


In [ ]:
#modeloPostnetNombre = '/content/drive/MyDrive/tesisPabloAxel/wavenet/modelos/wavenetReal/PostNetSimple12capas.pth'

In [ ]:
#postnetModel = PostNetSimple()
#postnetModel.load_state_dict(torch.load(modeloPostnetNombre))

In [ ]:
#wavenetModel = modelCargado
#WaveNet( layer_size=15, stack_size=2, in_channels=1, res_channels=128)
losses = []

In [19]:
loss_fn = nn.L1Loss()
learning_rate = 0.0015
optimizer = torch.optim.AdamW(params=wavenetModel.parameters(), lr=learning_rate)
#optimizerPost = torch.optim.AdamW(params=postnetModel.parameters(), lr=learning_rate)

mel_transform1 = transformsaudio.MelSpectrogram(sample_rate = 16000,
                                               n_fft = 2048,
                                                n_mels = 120,
                                                hop_length = 512).to('cuda')

mel_transform2 = transformsaudio.MelSpectrogram(sample_rate = 16000,
                                               n_fft = 512,
                                                n_mels = 80,
                                                hop_length = 128).to('cuda')


In [20]:
def prepareSpectogram(melspect):
    melspect = ((melspect - melspect.min()) / (melspect.max()-melspect.min()) + 0.00000001)
    melspect=melspect.log10()
    min_db = -6
    # Clamp the values in the tensor to be no less than min_db
    melspect = melspect.clamp(min=min_db)
    return melspect

In [21]:
def melspectogramLoss(y_pred, true_y):

  mel1true_y = (mel_transform1(true_y.to('cuda')))
  mel1y_pred = (mel_transform1(y_pred.to('cuda')))
  mel2true_y = (mel_transform2(true_y.to('cuda')))
  mel2y_pred = (mel_transform2(y_pred.to('cuda')))

  mel1true_y = prepareSpectogram(mel1true_y)
  mel1y_pred = prepareSpectogram(mel1y_pred)
  mel2true_y = prepareSpectogram(mel2true_y)
  mel2y_pred = prepareSpectogram(mel2y_pred)

  dif1 = (mel1true_y - mel1y_pred).abs().mean()
  dif2 = (mel2true_y - mel2y_pred).abs().mean()
  # Define the minimum dB value


  return dif1*15 , dif2*10

In [22]:
batch_size = 3

dataset = AudioCleaningDataset(directoryBase+'/CSV/audio_pocos.csv', directoryBase+'/audiosDivididos/', directoryBase+'/audiosDivididos/ruidosPocos/', directoryBase+"/audiosDivididos/reverbPocos")
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
melspectogramLoss(waveform, label)

In [24]:
losses = []

In [25]:
# %tensorboard --logdir=runs

In [26]:
NUM_EPOCHS = 10
for epoch in range(NUM_EPOCHS):
    avgLossOfBatch = []
    for i, (X, true_y) in enumerate(dataloader, 0):
        # zero gradients
        optimizer.zero_grad()
        # forward pass
        y_pred = wavenetModel(X.to("cuda"))
        #y_pred_postnet = postnetModel(y_pred)
        # calc losses

        lossAudio = loss_fn(y_pred, true_y) * 10
        lossSpect1, lossSpect2 = melspectogramLoss(y_pred, true_y)


        loss = lossAudio + lossSpect1 + lossSpect2
        #lossSpectPost1, lossSpectPost2 = melspectogramLoss(y_pred_postnet, true_y)
        #lossPostnet = loss_fn(y_pred_postnet, true_y) * 10 + lossSpectPost1 + lossSpectPost2
        # backward pass
        loss.backward()
        #lossPostnet.backward()
        # update weights
        optimizer.step()
        #optimizerPost.step()

        avgLossOfBatch.append(loss.item())
        xs = 2917 * epoch + i
        writer.add_scalar("Loss audio", lossAudio, xs)
        writer.add_scalar("Loss spect1", lossSpect1, xs)
        writer.add_scalar("Loss spect2", lossSpect2, xs)
        writer.add_scalar("Loss spect all", lossSpect1 + lossSpect2, xs)

        writer.add_scalar("Loss total", loss, i)

        if((i%1)==0):
          print(str(round(100*(i)/(2917/batch_size),2)) + "% del epoch " + str(epoch))
          prom = np.array(avgLossOfBatch).mean()
          losses.append(prom)
          avgLossOfBatch = []
          #print(prom)
          print("Perdida de wavenet: ")
          print(loss)
          print()
          print(lossAudio)
          print(lossSpect1)
          print(lossSpect2)
          #torch.save(wavenetModel.state_dict(), modeloNombre)
          torch.save(wavenetModel, modeloNombre)
            #torch.save(postnetModel.state_dict(), modeloPostnetNombre)


0.0% del epoch 0
Perdida de wavenet: 
tensor(2.1104, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0461, device='cuda:0', grad_fn=<MulBackward0>)
tensor(1.0223, device='cuda:0', grad_fn=<MulBackward0>)
tensor(1.0420, device='cuda:0', grad_fn=<MulBackward0>)
0.1% del epoch 0
Perdida de wavenet: 
tensor(1.7868, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0688, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.8075, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.9105, device='cuda:0', grad_fn=<MulBackward0>)
0.21% del epoch 0
Perdida de wavenet: 
tensor(2.0249, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0557, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.9928, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.9764, device='cuda:0', grad_fn=<MulBackward0>)
0.31% del epoch 0
Perdida de wavenet: 
tensor(1.7848, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0533, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.8570, device='cuda:0', grad_fn=<MulBackward0>)
te

3.29% del epoch 0
Perdida de wavenet: 
tensor(1.6412, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0514, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.7647, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.8251, device='cuda:0', grad_fn=<MulBackward0>)
3.39% del epoch 0
Perdida de wavenet: 
tensor(1.6589, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0686, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.8132, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.7770, device='cuda:0', grad_fn=<MulBackward0>)
3.5% del epoch 0
Perdida de wavenet: 
tensor(1.2273, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0350, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.5695, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.6229, device='cuda:0', grad_fn=<MulBackward0>)
3.6% del epoch 0
Perdida de wavenet: 
tensor(1.4976, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0698, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.7556, device='cuda:0', grad_fn=<MulBackward0>)
te

1.75% del epoch 1
Perdida de wavenet: 
tensor(1.4419, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0553, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.7082, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.6784, device='cuda:0', grad_fn=<MulBackward0>)
1.85% del epoch 1
Perdida de wavenet: 
tensor(1.6188, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0833, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.7639, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.7716, device='cuda:0', grad_fn=<MulBackward0>)
1.95% del epoch 1
Perdida de wavenet: 
tensor(1.8693, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0506, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.9045, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.9142, device='cuda:0', grad_fn=<MulBackward0>)
2.06% del epoch 1
Perdida de wavenet: 
tensor(1.6960, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0342, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.7708, device='cuda:0', grad_fn=<MulBackward0>)


0.21% del epoch 2
Perdida de wavenet: 
tensor(2.3279, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0540, device='cuda:0', grad_fn=<MulBackward0>)
tensor(1.1143, device='cuda:0', grad_fn=<MulBackward0>)
tensor(1.1597, device='cuda:0', grad_fn=<MulBackward0>)
0.31% del epoch 2
Perdida de wavenet: 
tensor(1.4563, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0452, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.7036, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.7075, device='cuda:0', grad_fn=<MulBackward0>)
0.41% del epoch 2
Perdida de wavenet: 
tensor(1.1120, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0409, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.5461, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.5249, device='cuda:0', grad_fn=<MulBackward0>)
0.51% del epoch 2
Perdida de wavenet: 
tensor(1.5341, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0812, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.7783, device='cuda:0', grad_fn=<MulBackward0>)


3.5% del epoch 2
Perdida de wavenet: 
tensor(1.7094, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0541, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.8921, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.7632, device='cuda:0', grad_fn=<MulBackward0>)
3.6% del epoch 2
Perdida de wavenet: 
tensor(1.7695, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0816, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.8127, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.8751, device='cuda:0', grad_fn=<MulBackward0>)
3.7% del epoch 2
Perdida de wavenet: 
tensor(1.9364, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0948, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.9334, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.9082, device='cuda:0', grad_fn=<MulBackward0>)
3.81% del epoch 2
Perdida de wavenet: 
tensor(1.6027, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0414, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.8809, device='cuda:0', grad_fn=<MulBackward0>)
ten

1.95% del epoch 3
Perdida de wavenet: 
tensor(1.9565, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0503, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.9953, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.9109, device='cuda:0', grad_fn=<MulBackward0>)
2.06% del epoch 3
Perdida de wavenet: 
tensor(1.8752, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0896, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.8405, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.9451, device='cuda:0', grad_fn=<MulBackward0>)
2.16% del epoch 3
Perdida de wavenet: 
tensor(1.7541, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0492, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.8503, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.8546, device='cuda:0', grad_fn=<MulBackward0>)
2.26% del epoch 3
Perdida de wavenet: 
tensor(1.5375, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0358, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.7583, device='cuda:0', grad_fn=<MulBackward0>)


0.41% del epoch 4
Perdida de wavenet: 
tensor(1.7510, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0864, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.8650, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.7995, device='cuda:0', grad_fn=<MulBackward0>)
0.51% del epoch 4
Perdida de wavenet: 
tensor(1.8829, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0689, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.9020, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.9120, device='cuda:0', grad_fn=<MulBackward0>)
0.62% del epoch 4
Perdida de wavenet: 
tensor(1.5645, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0658, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.6979, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.8008, device='cuda:0', grad_fn=<MulBackward0>)
0.72% del epoch 4
Perdida de wavenet: 
tensor(1.8828, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0546, device='cuda:0', grad_fn=<MulBackward0>)
tensor(1.0327, device='cuda:0', grad_fn=<MulBackward0>)


3.7% del epoch 4
Perdida de wavenet: 
tensor(1.0424, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0438, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.4802, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.5184, device='cuda:0', grad_fn=<MulBackward0>)
3.81% del epoch 4
Perdida de wavenet: 
tensor(1.3560, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0291, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.6627, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.6641, device='cuda:0', grad_fn=<MulBackward0>)
3.91% del epoch 4
Perdida de wavenet: 
tensor(1.3334, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0445, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.6762, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.6127, device='cuda:0', grad_fn=<MulBackward0>)
4.01% del epoch 4
Perdida de wavenet: 
tensor(1.2514, device='cuda:0', grad_fn=<AddBackward0>)

tensor(0.0663, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.6500, device='cuda:0', grad_fn=<MulBackward0>)
t

KeyboardInterrupt: 

In [ ]:
torch.save(wavenetModel.state_dict(), modeloNombre)

In [ ]:
13 % 1==0

In [ ]:
writer.close()


In [ ]:
#predicho = wavenetModel(X).detach().to("cuda")
X = X.to("cuda")
true_y = true_y.to("cuda")

In [ ]:
wdet = wavenetModel(X).detach()
predicho = wdet.cpu()
X = X.cpu()
true_y = true_y.cpu()

In [ ]:
opin = [1,2,4,5,6,7]

In [ ]:
audioAnalizar = 0

In [ ]:
lossSpect1, lossSpect2 = melspectogramLoss(predicho[audioAnalizar], true_y[audioAnalizar])
loss1 = loss_fn(predicho[audioAnalizar], true_y[audioAnalizar])
loss = loss1 + lossSpect1 + lossSpect2
print("error de prediccion " + str(loss.item()))


lossSpect1t, lossSpect2t = melspectogramLoss(X[audioAnalizar], true_y[audioAnalizar])
loss2 = loss_fn(X[audioAnalizar], true_y[audioAnalizar])
loss2t = loss2 + lossSpect1t + lossSpect2t

print("error sin hacer nada " + str(loss2t.item()))

In [ ]:
Audio(data=X.cpu()[audioAnalizar], rate=16000)

In [ ]:
Audio(data=predicho.cpu()[audioAnalizar], rate=16000)

In [ ]:
Audio(data=predichoPost.cpu()[audioAnalizar], rate=16000)

In [ ]:
Audio(data=true_y.cpu()[audioAnalizar], rate=16000)

In [ ]:
audioAnalizar = 1

In [ ]:
pos = 4200
delta = 2000
onda = predicho[audioAnalizar][0][pos:pos+delta]
onda2 = true_y[audioAnalizar][0][pos:pos+delta]
#onda3 = X[audioAnalizar][0][pos:pos+delta]

sns.lineplot(x=range(0,len(onda2)), y=(onda2), label="Real")
#sns.lineplot(x=range(0,len(onda3)), y=(onda3), label="Sucia")
sns.lineplot(x=range(0,len(onda)), y=(onda), label="Predicha")


In [ ]:
  mel1true_y = mel_transform1(true_y.to('cuda'))
  mel1y_pred = mel_transform1(predicho.to('cuda'))
  mel1sucio = mel_transform1(X.to('cuda'))


In [ ]:
mel_spectrogram_db = transformsaudio.AmplitudeToDB()(mel1true_y).cpu()


# Display the mel spectrogram
plt.figure(figsize=(10, 4))
plt.imshow(mel_spectrogram_db[0][0].detach().numpy(), cmap='viridis', origin='lower', aspect='auto')
plt.colorbar(format="%+2.0f dB")
plt.title("Mel Spectrogram verdadero")
plt.xlabel("Time")
plt.ylabel("Mel Frequency Bin")
plt.show()

In [ ]:
mel_spectrogram_db2 = transformsaudio.AmplitudeToDB()(mel1y_pred).cpu()


# Display the mel spectrogram
plt.figure(figsize=(10, 4))
plt.imshow(mel_spectrogram_db2[0][0].detach().numpy(), cmap='viridis', origin='lower', aspect='auto')
plt.colorbar(format="%+2.0f dB")
plt.title("Mel Spectrogram predicho")
plt.xlabel("Time")
plt.ylabel("Mel Frequency Bin")
plt.show()

In [ ]:
mel_spectrogram_db3 = transformsaudio.AmplitudeToDB()(mel1sucio).cpu()


# Display the mel spectrogram
plt.figure(figsize=(10, 4))
plt.imshow(mel_spectrogram_db3[0][0].detach().numpy(), cmap='viridis', origin='lower', aspect='auto')
plt.colorbar(format="%+2.0f dB")
plt.title("Mel Spectrogram sucio")
plt.xlabel("Time")
plt.ylabel("Mel Frequency Bin")
plt.show()

In [ ]:
audioAnalizar = 0

In [ ]:
import soundfile as sf


In [ ]:
for audioAnalizar in opin:
  sf.write('sucio'+str(audioAnalizar)+'.wav', np.ravel(X[audioAnalizar]), 16000, 'PCM_24')
  sf.write('original'+str(audioAnalizar)+'.wav', np.ravel(true_y[audioAnalizar]), 16000, 'PCM_24')
  sf.write('limpiado'+str(audioAnalizar)+'.wav', np.ravel(predicho[audioAnalizar][0]), 16000, 'PCM_24')

In [ ]:
sf.write('sucio'+str(audioAnalizar)+'.wav', np.ravel(X[audioAnalizar]), 16000, 'PCM_24')

In [ ]:
sf.write('limpiadoEco'+str(audioAnalizar)+'.wav', np.ravel(predicho[audioAnalizar][0]), 16000, 'PCM_24')

In [ ]:
sf.write('originalSinTocarWavenetReplica'+str(audioAnalizar)+'.wav', np.ravel(true_y[audioAnalizar]), 16000, 'PCM_24')

In [ ]:
#dataloader give 1 example
ini = time.time()
for waveform, label in dataloader:
    print(waveform.shape)
    print(label.shape)
    break
fin = time.time()

In [ ]:
out = modelPanoramico(waveform)

In [ ]:
nuevow = WaveNet( layer_size=18, stack_size=2, in_channels=1, res_channels=128)

In [ ]:
#nuevow.load_state_dict(torch.load(modeloNombre), map_location=torch.device('cpu'))
nuevow = torch.load(modeloNombre, map_location=torch.device('cpu'))


In [ ]:
predicho = nuevow(waveform).detach().cpu()
waveform = waveform.cpu().detach()
true_y = label.cpu()
X = waveform

In [ ]:
lossSpect1, lossSpect2 = melspectogramLoss(predicho[audioAnalizar], true_y[audioAnalizar])
loss1 = loss_fn(predicho[audioAnalizar], true_y[audioAnalizar])
loss = loss1 + lossSpect1 + lossSpect2
print("error de prediccion " + str(loss.item()))

lossSpect1t, lossSpect2t = melspectogramLoss(waveform[audioAnalizar], true_y[audioAnalizar])
loss2 = loss_fn(waveform[audioAnalizar], true_y[audioAnalizar])
loss2t = loss2 + lossSpect1t + lossSpect2t

print("error sin hacer nada " + str(loss2t.item()))

In [ ]:
class WaveNetPanoramico(pl.LightningModule):
    def __init__(self, layer_size, stack_size, in_channels, res_channels):
        """
        Stack residual blocks by layer and stack size
        :param layer_size: integer, 10 = layer[dilation=1, dilation=2, 4, 8, 16, 32, 64, 128, 256, 512]
        :param stack_size: integer, 5 = stack[layer1, layer2, layer3, layer4, layer5]
        :param in_channels: number of channels for input data. skip channel is same as input channel
        :param res_channels: number of residual channel for input, output
        :return:
        """
        super(WaveNetPanoramico, self).__init__()

        self.receptive_fields = self.calc_receptive_fields(layer_size, stack_size)

        self.causal = CausalConv1d(in_channels, res_channels)

        self.res_stack = ResidualStack(layer_size, stack_size, res_channels, in_channels)

        self.densnet = DensNet(in_channels)

        self.loss_fun = nn.MSELoss()
        inp = layer_size*stack_size+res_channels+1
        inp2 = math.floor(inp/2)
        self.convFinal1 = nn.Conv1d(in_channels=layer_size*stack_size+res_channels+1,
                                   out_channels=inp2,
                                   kernel_size=15, stride=1,
                                   dilation=1, padding=7, bias=True).to("cuda")
        self.convFinal2 = nn.Conv1d(in_channels=inp2,
                                   out_channels=1,
                                   kernel_size=15, stride=1,
                                   dilation=1, padding=7, bias=True).to("cuda")

        self.convFinal3 = nn.Conv1d(in_channels=1,
                                   out_channels=1,
                                   kernel_size=101, stride=1,
                                   dilation=1, padding=50, bias=False).to("cuda")


        self.relu = nn.ReLU()

    @staticmethod
    def calc_receptive_fields(layer_size, stack_size):
        layers = [2 ** i for i in range(0, layer_size)] * stack_size
        num_receptive_fields = np.sum(layers)

        return int(num_receptive_fields)

    def calc_output_size(self, x):
        output_size = int(x.size(2)) - self.receptive_fields

        #self.check_input_size(x, output_size)

        return inputSize

    def check_input_size(self, x, output_size):
        if output_size < 1:
            raise InputSizeError(int(x.size(2)), self.receptive_fields, output_size)

    def forward(self, x):
        """
        The size of timestep(3rd dimention) has to be bigger than receptive fields
        :param x: Tensor[batch, timestep, channels]
        :return: Tensor[batch, timestep, channels]
        """
        output = x#.transpose(1, 2)
        copiaConvolucionFinal = x.clone()

        output_size = self.calc_output_size(output)

        output = self.causal(output)

        skip_connections = self.res_stack(output, output_size)



        #en vez de sumar skip connections hago convolucion
        #print(skip_connections.shape)
        #output = torch.sum(skip_connections, dim=0)
        skip_connections_squeezed = skip_connections.squeeze()
        # Check the shape
        # Swap the first two dimensions
        skip_connections_squeezed = skip_connections_squeezed.transpose(0, 1)
        output = torch.cat((output, copiaConvolucionFinal), dim=1)

        output = torch.cat((output, skip_connections_squeezed), dim=1)

        output = self.convFinal1(output)
        output = self.relu(output)
        output = self.convFinal2(output)
        #output = self.relu(output)
        #output = self.convFinal3(output)

        #output = self.densnet(output)

        return output#.transpose(1, 2).contiguous()

In [ ]:
class PostNetSimple(pl.LightningModule):
  def __init__(self, layers=12):
    super(PostNetSimple, self).__init__()
    self.convInicial = nn.Conv1d(in_channels=1,
                            out_channels=128,
                            kernel_size=33, stride=1,
                            dilation=1, padding=16, bias=True).to("cuda")
    self.totalLayers = layers
    self.convs = []
    for conv in range(0, layers):
      self.convs.append(
          nn.Conv1d(in_channels=128,
                            out_channels=128,
                            kernel_size=33, stride=1,
                            dilation=1, padding=16, bias=True).to("cuda")
      )
    self.convFinal = nn.Conv1d(in_channels=129,
                            out_channels=1,
                            kernel_size=33, stride=1,
                            dilation=1, padding=16, bias=True).to("cuda")
    self.tan = nn.Tanh()

  def forward(self, x):
    xCopia = x.clone()
    x = self.convInicial(x)
    x = self.tan(x)
    for conv in self.convs:
      x = conv(x)
      x = self.tan(x)
    x = torch.cat((x, xCopia), dim=1)
    x = self.convFinal(x)
    return x